dane = {'blue_more': 0, 
        'sabres': 0, 
        'jokers': 0, 
        'gold': 3, 
        'extra_money_fivecards': 0, 
        'black_more': 0, 
        'points': 0, 
        'extra_card': 0, 
        'expeditions': 0, 
        'red_more': 0, 
        'anchors': 0, 
        'green_more': 0, 
        'crosses': 0, 
        'extra_money_notrade': 0, 
        'yellow_more': 0, 
        'houses': 0, 
        'pay_less': 0
       }

In [166]:
import random
import json
from IPython.display import Image, clear_output

In [178]:
class Settings(object):
    def __init__(self):
        self.settings = self.__read_to_tuples__('game_settings.ini')
        
    def __read_file__(self, filename):
        '''Function for read files to variables.'''
        with open(filename, 'r') as file_input:
            return json.load(file_input)
    
    def __tuple_inception__(self, data_set):
        '''Function for change type of read data sets as tuples.'''
        if type(data_set) == dict:
            for key, val in data_set.items():
                if type(val) == list:
                    data_set[key] = tuple(val)
        elif type(data_set) == list:
            for lst in data_set:
                for idx in range(len(lst)):
                    if type(lst[idx]) == list:
                        lst[idx] = tuple(lst[idx])
                data_set[data_set.index(lst)] = tuple(lst)
            data_set = tuple(data_set)
        return data_set
    
    def __read_to_tuples__(self, filename):
        '''Connection both above functions, better readable code.'''
        return self.__tuple_inception__(self.__read_file__(filename))

In [184]:
class Talia(object):
    def __init__(self):
#         with open('deck_portroyal.txt', 'r') as f_in:
#             self.deck = json.load(f_in) #trzyma cala talie
        deck_file = 'pr_deck.txt'
        self.deck = Settings().__read_to_tuples__(deck_file)
        self.choosen = [] # catch choosen cards
        self.expeditions = [] # catch expeditions laying on table
                
        random.shuffle(self.choosen)

    def losuj_karte(self):
        '''Funkcja losujaca karte z talii'''
        #self.losowanie = random.choice(tuple(deck.items()))[1]
        self.losowanie = random.choice(self.deck)
        print(self.losowanie)
        if self.losowanie not in (self.choosen, self.expeditions):
            if self.losowanie[0] == 'expedition':
                self.expeditions.append(self.losowanie)
                return self.losuj_karte()
            else:
                self.choosen.append(self.losowanie)
                return Card(self.losowanie)
        else:
            return self.losuj_karte()
        
    def resetuj_talie(self):
        '''Czysci liste wylosowanych kart i kart ekspedycji lezacych na stole'''
        self.choosen = []
        self.expeditions = []

In [185]:
class Card(object):
    def __init__(self, card_data):
        '''Set variables for choosen card.'''
        ### ---> change on name depend from choosen lang
        self.name = card_data[0][0]
        self.name_pl = card_data[0][1]
        self.color = card_data[1]
        self.points = card_data[2]
        self.cost = card_data[3]
        self.feature = card_data[4]
        self.nr = card_data[5]

In [186]:
class Gracz(object):
    def __init__(self, id):
        self.gid = id
        self.karta = None
        self.reka = []
        with open('data_dict.txt', 'r') as f_in:
            self.dane = json.load(f_in) #trzyma dane dla danego gracza, ilosc mieczy, zlota, etc
        
    def podnies_karte(self, karta):
        '''Podnosi karte ze zbioru dostepnych kart'''
        self.karta = karta
        
    def wez_karte(self):
        '''Bierze karte do siebie - wchodzi w posiadanie danej karty'''
        self.reka.append(self.karta)
        
    def karty_w_rece(self):
        return self.reka
        
    def pokaz_karte(self):
        '''Pokazuje dane karty'''
        return (self.karta.name, self.karta.name_pl, self.karta.color, self.karta.points, self.karta.cost, self.karta.feature, self.karta.nr)
    
    def wynik_gracza(self):
        '''Pokazuje wynik gracza w postaci ilosci punktow, ilosci ekspedycji'''
        return (self.dane['points'], self.dane['expeditions'])
    
    def statystyki(self):
        return self.dane
    
    def dorysuj_linie(self, co_str, dane_str):
        ilosc = self.long - (len(co_str) + len(str(dane_str)) + self.long-(self.long-3))
        return '\n| ' + co_str + ': ' + str(dane_str) + ' '*(ilosc) + '|'
    
    def rysuj_karte(self):
        '''Rysuje karte'''
        opisy = {'Kolor':self.karta.color, 'Punkty':self.karta.points, 'Koszt':self.karta.cost, 
                 'Cecha':self.karta.feature, 'Nazwa karty':self.karta.name_pl, 'Numer karty':self.karta.nr}
        self.long = 32
        up = ' ' + '_'*self.long
        dn = '|' + '_'*self.long + '|'
        mid = '|' + ' '*self.long + '|'
        mid_all = ''
        for key, val in opisy.items():
            if val != None:
                mid_all += self.dorysuj_linie(key, val)
            elif val == None:
                mid_all += self.dorysuj_linie(key, '-')
            else:
                mid_all += mid
        output = '\n'.join(map(str, (up, mid, mid_all.lstrip(), mid, mid, mid, mid, mid, mid, mid, dn)))
        return output      

In [187]:
class Rozgrywka(object):
    def __init__(self, ilosc_graczy):
        self.sets = Settings().settings
        self.lang = self.sets['choosen_language']
        self.gracze = [Gracz(gid) for gid in range(ilosc_graczy)]
        self.imiona = {}
        self.talia = Talia()
        self.odrzucone = [] #lista kart odrzuconych przez graczy w czasie gry - np. statki, etc.
        self.stol = [] #lista kart wylozonych w pierwszej fazie gry przez danego gracza
        self.numery_stol = {} #numery kart znajdujacych sie na stole KEY:VAL -> Index:Nr
        self.karta_nr = {} #trzyma obiekt nr_karty:Karta
        self.str_nr = [str(idx) for idx in range(121)] #stringi z numerow do porownywania
        self.wziete = [] #nr_kart ktore zabral gracz i ktore sa juz niedostepne
        
    def _przydziel_karte(self, gracz):
        '''Funkcja przydziela graczowi karte wylosowana z talii'''
        karta = self.talia.losuj_karte()
        gracz.podnies_karte(karta)
        
    def _karta_ze_stolu(self, gracz, idx):
        '''Funkcja pokazuje karty gracza, ktore leza na stole'''
        karta = self.stol[idx]
        gracz.podnies_karte(karta)
        
    def _wylon_zwyciezce(self):
        '''Szukamy zwyciezcy porownujac karty kazdego gracza'''
        wygrany_zestaw = max(self.gracze, key=lambda g: g.wynik_gracza())
        return wygrany_zestaw
        
    def _oglos_zwyciezce(self, zwyciezca):
        '''Oglaszamy zwyciezkiego gracza'''
        wygrana_karta = zwyciezca.wynik_gracza()
        print('Wygral gracz o id {} majac: {}\n'.format(zwyciezca.gid, wygrana_karta))
        #print(zwyciezca.rysuj_karte())
        #return Image(url='http://ct.fra.bz/ol/fz/sw/i46/5/12/20/fbz_4e64b1eab5e56a0ee0cd125d70727ea5.jpg', width=200, height=200)
        return zwyciezca

    def _poznaj_graczy(self):
        '''Funkcja przyjmuje imiona graczy i zapisuje do slownika'''
        for g in range(len(self.gracze)):
            self.imiona[self.gracze[g]] = input(self.sets['hello_name'][self.lang].format(g))
        print('Milo mi was zaprosic do gry PR:')
        for imie in sorted(self.imiona.values()):
            print(imie)
            
    def _karty_na_stol(self, gracz):
        '''Funkcja kieruje czescia gry odpowiedzialna za losowanie kart i wyswietlanie stanu stolu'''
        clear_output()
        self._przydziel_karte(gracz)
        self.stol.append(gracz.karta)
        self.karta_nr[gracz.pokaz_karte()[-1]] = gracz.karta
        print('Karty na stole: {}\n'.format(len(self.stol)))
        for kns in range(len(self.stol)):
            self._karta_ze_stolu(gracz, kns)
            # Tworzy slownik z kluczami odpowiadajacymi indeksom kart lezacym na stole
            # i wartosciami bedacymi numerem id karty
            self.numery_stol[kns] = gracz.pokaz_karte()[-1]
            print(gracz.rysuj_karte())
            if gracz.pokaz_karte()[0] == 'ship':
                if gracz.pokaz_karte()[-2] != 0:
                    if gracz.statystyki()['sabres']+7 >= gracz.pokaz_karte()[-2]:
                        if input('Czy chcesz odeprzec statek? T/[n] ').lower() in self.sets['yes_answers']:
                            self.stol.remove(self.karta_nr[gracz.pokaz_karte()[-1]])
                            self.numery_stol_tmp = self.numery_stol.copy()
                            for key, val in self.numery_stol_tmp.items():
                                if val == gracz.pokaz_karte()[-1]:
                                    self.numery_stol.pop(key)
            if len(self.talia.expeditions) > 0:
                print('Sa dostepne ekspedycje')
        while input('Losowac karte? T/[n] ').lower() in self.sets['yes_answers']:
            self._karty_na_stol(gracz)
        else:
            pass
                
    def _karty_z_reki(self, gracz):
        '''Funkcja pokazuje karty jakie posiada gracz'''
        print('Karty, ktore posiadasz')
        print(gracz.karty_w_rece())
        for idx in gracz.karty_w_rece():
            gracz.podnies_karte(idx)
            print(gracz.rysuj_karte())
            
    def _odrzuc_karty(self):
        '''Czysci stol'''
        self.odrzucone += self.stol.copy()
        self.stol = []
        self.numery_stol = {}
        self.karta_nr = {}
        self.wziete = []
        
    def _pokaz_dostepne(self):
        '''Pokazuje numery kart lezacych na stole'''
        print('Numery dostepnych kart:')
        for val in self.numery_stol.values():
             print(val)
        
    def _wybor_kart(self, gracz):
        '''Tutaj gracz wybiera, ktore karty ze stolu chce kupic badz sprzedac'''
        stats = gracz.statystyki()
        ile_kart = 2 + stats['extra_card'] #ile kart moze zabrac gracz
        self._pokaz_dostepne()
        print('Wpisz numer karty ktora chcesz sprzedac lub kupic. Jesli chcesz zakonczyc faze handlu - wcsnij enter')
        while ile_kart > 0 and len(self.stol) > 0:
            print('Mozesz wziac jeszcze {} kart'.format(ile_kart))
            wybor = input('Numer lub Enter: ')
            if wybor in self.str_nr:
                if int(wybor) in self.numery_stol.values() and int(wybor) not in self.wziete:
                    for key, val in self.numery_stol.items():
                        if int(wybor) == self.numery_stol[key]:
                            gracz.podnies_karte(self.karta_nr[int(wybor)])
                            if gracz.pokaz_karte()[0] == 'ship':
                                stats['gold'] += gracz.pokaz_karte()[4]
                            else:
                                how_much = gracz.pokaz_karte()[-3] - stats['pay_less']
                                if how_much >= stats['gold']:
                                    stats['gold'] -= how_much
                            self._karta_ze_stolu(gracz, key)
                            gracz.wez_karte()
                            print('Wzieto karte numer {}'.format(wybor))
                            self.wziete.append(self.numery_stol[key])
                            self.stol.remove(self.karta_nr[int(wybor)])
                            ile_kart -= 1
                else:
                    print('Zla wartosc. Podaj jeszcze raz')
            else:
                print('Zakonczenie fazy handlu')
                ile_kart = 0
    
    def graj(self):
        self.talia.resetuj_talie() # Na poczatku kazdej gry resetuj talie
        self._poznaj_graczy()
        for g in self.gracze:
            print('Kolejka gracza {}.'.format(self.imiona[g]))
            self._karty_na_stol(g)
            if len(self.stol) > 0:
                print('Zakonczono faze dobierania kart. Rozpoczyna sie faza handlu')
                self._wybor_kart(g)
            else:
                print('Zakonczono faze dobierania kart. Ze wzgledu na brak kart na stole - faza handlu zostaje pominieta')
            self._odrzuc_karty()
            print('---karty gracza {} ---'.format(self.imiona[g]))
            self._karty_z_reki(g)
            print(g.statystyki())
            print('karta_nr - na stole')
            print(self.karta_nr)
            print('karty na stole')
            print(self.numery_stol)
            print('karty na stole - self.stol')
            print(self.stol)
            print('karty w hasioku')
            print(self.odrzucone)
            

#         zwyciezca = self._wylon_zwyciezce()
#         return self._oglos_zwyciezce(zwyciezca)

In [188]:
r = Rozgrywka(ilosc_graczy=1)
r.graj()

(('ship', 'niebieska fluita'), 'blue', None, 3, ('sabres', 2), 87)
Karty na stole: 4

 ________________________________
|                                |
| Cecha: ('anchors', 1)          |
| Kolor: -                       |
| Koszt: 4                       |
| Punkty: 1                      |
| Numer karty: 18                |
| Nazwa karty: kapitan           |
|                                |
|                                |
|                                |
|                                |
|                                |
|                                |
|                                |
|________________________________|
 ________________________________
|                                |
| Cecha: ('sabres', 1)           |
| Kolor: -                       |
| Koszt: 3                       |
| Punkty: 1                      |
| Numer karty: 35                |
| Nazwa karty: żeglarz           |
|                                |
|                                |
|     

TypeError: unorderable types: int() >= tuple()

In [11]:
ns={'a':1,'b':2}
bs=ns.copy()
ns['c']=3
print(bs)

{'b': 2, 'a': 1}


In [165]:
import random
aaa=[1,2,3,4,5]
random.shuffle(aaa)
print(aaa)

[3, 4, 5, 1, 2]


In [ ]:
heh = []
for idx in range(121):
    heh.append(str(idx))

In [ ]:
for i in range(10): print(i)

In [ ]:
dane1 = {'sabres':0,
                     'houses':0,
                     'anchors':0,
                     'crosses':0,
                     'jokers':0,
                     'yellow_more':0,
                     'blue_more':0,
                     'green_more':0,
                     'red_more':0,
                     'black_more':0,
                     'pay_less':0,
                     'extra_money_notrade':0,
                     'extra_money_fivecards':0,
                     'extra_card':0,
                     'points':0,
                     'expeditions':0,
                     'gold':0
                    }

import json
with open('data_dict.txt', 'w') as f:
    json.dump(dane1, f)

dane2 = {'sabres':1,
                     'houses':10,
                     'anchors':10,
                     'crosses':10,
                     'jokers':10,
                     'yellow_more':10,
                     'blue_more':10,
                     'green_more':0,
                     'red_more':10,
                     'black_more':0,
                     'pay_less':0,
                     'extra_money_notrade':0,
                     'extra_money_fivecards':10,
                     'extra_card':11,
                     'points':110,
                     'expeditions':0,
                     'gold':0
                    }

for key,val in dane2.items():
    dane1[key] += val
dane1

In [ ]:
dane2 = {'sabres':1,
                     'houses':10,
                     'anchors':10,
                     'crosses':10,
                     'jokers':10,
                     'yellow_more':10,
                     'blue_more':10,
                     'green_more':0,
                     'red_more':10,
                     'black_more':0,
                     'pay_less':0,
                     'extra_money_notrade':0,
                     'extra_money_fivecards':10,
                     'extra_card':11,
                     'points':110,
                     'expeditions':0,
                     'gold':0
                    }
dane2.pop('anchors')

In [ ]:
dane2

In [ ]:
litr={3:81,5:135,10:265,20:527}
for k,v in sorted(litr.items()):
    print('Przy {} litrach za {} zlotych -> 1 litr - {} zlotych'.format(k,v,v/k))

In [ ]:
0.15*20

%%writefile trabant.txt
trabant

In [50]:
settings = {'language':{'pol':0, 'eng':1},
                         'choosen_language':0,
                         'deck_file':'deck_portroyal.txt',
                         'data_file':'data_dict.txt',
                         'hello_name':('Witaj Gracz{}. Podaj swoje imie: ', 'Hello Player{}. Type your name: '),
                         'yes_answers':('t', 'tak', 'y', 'yes')
                        }
print(settings)
with open('game_settings.ini', 'w') as f:
    json.dump(settings, f)

{'choosen_language': 0, 'data_file': 'data_dict.txt', 'yes_answers': ('t', 'tak', 'y', 'yes'), 'deck_file': 'deck_portroyal.txt', 'hello_name': ('Witaj Gracz{}. Podaj swoje imie: ', 'Hello Player{}. Type your name: '), 'language': {'pol': 0, 'eng': 1}}


In [147]:
%%sh
tail -20 pr_deck.txt

[[["vendor", "handlarz"], "yellow", 1, 3, ["yellow_more", 1], 1], [["vendor", "handlarz"], "yellow", 2, 5, ["yellow_more", 1], 2], [["vendor", "handlarz"], "blue", 1, 3, ["blue_more", 1], 3], [["vendor", "handlarz"], "blue", 2, 5, ["blue_more", 1], 4], [["vendor", "handlarz"], "green", 1, 3, ["green_more", 1], 5], [["vendor", "handlarz"], "green", 1, 3, ["green_more", 1], 6], [["vendor", "handlarz"], "red", 1, 3, ["red_more", 1], 7], [["vendor", "handlarz"], "red", 1, 3, ["red_more", 1], 8], [["vendor", "handlarz"], "black", 1, 3, ["black_more", 1], 9], [["vendor", "handlarz"], "black", 1, 3, ["black_more", 1], 10], [["settler", "osadnik"], null, 1, 4, ["houses", 1], 11], [["settler", "osadnik"], null, 1, 4, ["houses", 1], 12], [["settler", "osadnik"], null, 1, 4, ["houses", 1], 13], [["settler", "osadnik"], null, 1, 4, ["houses", 1], 14], [["settler", "osadnik"], null, 1, 4, ["houses", 1], 15], [["captain", "kapitan"], null, 1, 4, ["anchors", 1], 16], [["captain", "kapitan"], null, 1,

In [11]:
with open('deck_portroyal.txt', 'r') as f:
    deck4 = json.load(f)
deeck = []
#print(deck4)
for k,v in sorted(deck4.items()):
    deeck.append(tuple(v))

print(tuple(deeck))
# with open('deck_portroyal.txt', 'w') as f:
#     json.dump(deeck, f)

(('vendor', 'handlarz', 'yellow', 1, 3, 12, 1), ('vendor', 'handlarz', 'black', 1, 3, 12, 10), ('ship', 'zielony skiff', 'green', None, 4, 5, 100), ('ship', 'czerwona fregata', 'red', None, 1, 1, 101), ('ship', 'czerwona fregata', 'red', None, 1, 1, 102), ('ship', 'czerwona fregata', 'red', None, 1, 1, 103), ('ship', 'czerwona fregata', 'red', None, 2, 3, 104), ('ship', 'czerwona fregata', 'red', None, 2, 3, 105), ('ship', 'czerwona fregata', 'red', None, 2, 3, 106), ('ship', 'czerwona fregata', 'red', None, 3, 6, 107), ('ship', 'czerwona fregata', 'red', None, 3, 6, 108), ('ship', 'czerwona fregata', 'red', None, 3, 0, 109), ('settler', 'osadnik', None, 1, 4, 8, 11), ('ship', 'czerwona fregata', 'red', None, 4, 0, 110), ('ship', 'czarny galeon', 'black', None, 1, 2, 111), ('ship', 'czarny galeon', 'black', None, 1, 2, 112), ('ship', 'czarny galeon', 'black', None, 1, 2, 113), ('ship', 'czarny galeon', 'black', None, 2, 4, 114), ('ship', 'czarny galeon', 'black', None, 2, 4, 115), ('sh

In [138]:
decker = []
decker.append((('vendor', 'handlarz'), 'yellow', 1, 3, ('yellow_more', 1), 1))
decker.append((('vendor', 'handlarz'), 'yellow', 2, 5, ('yellow_more', 1), 2))
decker.append((('vendor', 'handlarz'), 'blue', 1, 3, ('blue_more', 1), 3))
decker.append((('vendor', 'handlarz'), 'blue', 2, 5, ('blue_more', 1), 4))
decker.append((('vendor', 'handlarz'), 'green', 1, 3, ('green_more', 1), 5))
decker.append((('vendor', 'handlarz'), 'green', 1, 3, ('green_more', 1), 6))
decker.append((('vendor', 'handlarz'), 'red', 1, 3, ('red_more', 1), 7))
decker.append((('vendor', 'handlarz'), 'red', 1, 3, ('red_more', 1), 8))
decker.append((('vendor', 'handlarz'), 'black', 1, 3, ('black_more', 1), 9))
decker.append((('vendor', 'handlarz'), 'black', 1, 3, ('black_more', 1), 10))
decker.append((('settler', 'osadnik'), None, 1, 4, ('houses', 1), 11))
decker.append((('settler', 'osadnik'), None, 1, 4, ('houses', 1), 12))
decker.append((('settler', 'osadnik'), None, 1, 4, ('houses', 1), 13))
decker.append((('settler', 'osadnik'), None, 1, 4, ('houses', 1), 14))
decker.append((('settler', 'osadnik'), None, 1, 4, ('houses', 1), 15))
decker.append((('captain', 'kapitan'), None, 1, 4, ('anchors', 1), 16))
decker.append((('captain', 'kapitan'), None, 1, 4, ('anchors', 1), 17))
decker.append((('captain', 'kapitan'), None, 1, 4, ('anchors', 1), 18))
decker.append((('captain', 'kapitan'), None, 1, 4, ('anchors', 1), 19))
decker.append((('captain', 'kapitan'), None, 1, 4, ('anchors', 1), 20))
decker.append((('priest', 'kapłan'), None, 1, 4, ('crosses', 1), 21))
decker.append((('priest', 'kapłan'), None, 1, 4, ('crosses', 1), 22))
decker.append((('priest', 'kapłan'), None, 1, 4, ('crosses', 1), 23))
decker.append((('priest', 'kapłan'), None, 1, 4, ('crosses', 1), 24))
decker.append((('priest', 'kapłan'), None, 1, 4, ('crosses', 1), 25))
decker.append((('handyman', 'złota rączka'), None, 1, 6, ('jokers', 1), 26))
decker.append((('handyman', 'złota rączka'), None, 1, 6, ('jokers', 1), 27))
decker.append((('handyman', 'złota rączka'), None, 1, 6, ('jokers', 1), 28))
decker.append((('sailor', 'żeglarz'), None, 1, 3, ('sabres', 1), 29))
decker.append((('sailor', 'żeglarz'), None, 1, 3, ('sabres', 1), 30))
decker.append((('sailor', 'żeglarz'), None, 1, 3, ('sabres', 1), 31))
decker.append((('sailor', 'żeglarz'), None, 1, 3, ('sabres', 1), 32))
decker.append((('sailor', 'żeglarz'), None, 1, 3, ('sabres', 1), 33))
decker.append((('sailor', 'żeglarz'), None, 1, 3, ('sabres', 1), 34))
decker.append((('sailor', 'żeglarz'), None, 1, 3, ('sabres', 1), 35))
decker.append((('sailor', 'żeglarz'), None, 2, 5, ('sabres', 1), 36))
decker.append((('sailor', 'żeglarz'), None, 2, 5, ('sabres', 1), 37))
decker.append((('sailor', 'żeglarz'), None, 3, 7, ('sabres', 1), 38))
decker.append((('pirate', 'pirat'), None, 1, 5, ('sabres', 2), 39))
decker.append((('pirate', 'pirat'), None, 2, 7, ('sabres', 2), 40))
decker.append((('pirate', 'pirat'), None, 3, 9, ('sabres', 2), 41))
decker.append((('madmoiselle', 'madmoiselle'), None, 2, 7, ('pay_less', -1), 42))
decker.append((('madmoiselle', 'madmoiselle'), None, 2, 7, ('pay_less', -1), 43))
decker.append((('madmoiselle', 'madmoiselle'), None, 3, 9, ('pay_less', -1), 44))
decker.append((('madmoiselle', 'madmoiselle'), None, 3, 9, ('pay_less', -1), 45))
decker.append((('jester', 'błazen'), None, 1, 5, ('extra_money_notrade', 1), 46))
decker.append((('jester', 'błazen'), None, 2, 7, ('extra_money_notrade', 1), 47))
decker.append((('jester', 'błazen'), None, 2, 7, ('extra_money_notrade', 1), 48))
decker.append((('jester', 'błazen'), None, 2, 7, ('extra_money_notrade', 1), 49))
decker.append((('jester', 'błazen'), None, 3, 9, ('extra_money_notrade', 1), 50))
decker.append((('admiral', 'admirał'), None, 1, 5, ('extra_money_fivecards', 2), 51))
decker.append((('admiral', 'admirał'), None, 2, 7, ('extra_money_fivecards', 2), 52))
decker.append((('admiral', 'admirał'), None, 2, 7, ('extra_money_fivecards', 2), 53))
decker.append((('admiral', 'admirał'), None, 2, 7, ('extra_money_fivecards', 2), 54))
decker.append((('admiral', 'admirał'), None, 3, 9, ('extra_money_fivecards', 2), 55))
decker.append((('admiral', 'admirał'), None, 3, 9, ('extra_money_fivecards', 2), 56))
decker.append((('governor', 'gubernator'), None, 0, 8, ('extra_card', 1), 57))
decker.append((('governor', 'gubernator'), None, 0, 8, ('extra_card', 1), 58))
decker.append((('governor', 'gubernator'), None, 0, 8, ('extra_card', 1), 59))
decker.append((('governor', 'gubernator'), None, 0, 8, ('extra_card', 1), 60))
decker.append((('expedition', 'ekspedycja'), None, 4, None, ('houses', 2), 61))
decker.append((('expedition', 'ekspedycja'), None, 4, None, ('anchors', 2), 62))
decker.append((('expedition', 'ekspedycja'), None, 4, None, ('crosses', 2), 63))
decker.append((('expedition', 'ekspedycja'), None, 6, None, ('crosses_houses', 3), 64))
decker.append((('expedition', 'ekspedycja'), None, 6, None, ('anchors_houses', 3), 65))
decker.append((('expedition', 'ekspedycja'), None, 5, None, ('houses_anchors_crosses', 3), 66))
decker.append((('tax', 'podwyżka podatków'), None, None, None, ('tax_sabres', 1), 67))
decker.append((('tax', 'podwyżka podatków'), None, None, None, ('tax_sabres', 1), 68))
decker.append((('tax', 'podwyżka podatków'), None, None, None, ('tax_points', 1), 69))
decker.append((('tax', 'podwyżka podatków'), None, None, None, ('tax_points', 1), 70))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 1, ('sabres', 1), 71))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 1, ('sabres', 1), 72))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 1, ('sabres', 1), 73))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 2, ('sabres', 1), 74))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 2, ('sabres', 2), 75))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 2, ('sabres', 2), 76))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 3, ('sabres', 2), 77))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 3, ('sabres', 4), 78))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 3, ('sabres', 4), 79))
decker.append((('ship', 'żółta szalupa'), 'yellow', None, 4, ('sabres', 4), 80))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 1, ('sabres', 1), 81))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 1, ('sabres', 1), 82))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 1, ('sabres', 1), 83))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 2, ('sabres', 1), 84))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 2, ('sabres', 2), 85))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 2, ('sabres', 2), 86))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 3, ('sabres', 2), 87))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 3, ('sabres', 5), 88))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 3, ('sabres', 5), 89))
decker.append((('ship', 'niebieska fluita'), 'blue', None, 4, ('sabres', 5), 90))
decker.append((('ship', 'zielony skiff'), 'green', None, 1, ('sabres', 1), 91))
decker.append((('ship', 'zielony skiff'), 'green', None, 1, ('sabres', 1), 92))
decker.append((('ship', 'zielony skiff'), 'green', None, 1, ('sabres', 1), 93))
decker.append((('ship', 'zielony skiff'), 'green', None, 2, ('sabres', 1), 94))
decker.append((('ship', 'zielony skiff'), 'green', None, 2, ('sabres', 3), 95))
decker.append((('ship', 'zielony skiff'), 'green', None, 2, ('sabres', 3), 96))
decker.append((('ship', 'zielony skiff'), 'green', None, 3, ('sabres', 3), 97))
decker.append((('ship', 'zielony skiff'), 'green', None, 3, ('sabres', 5), 98))
decker.append((('ship', 'zielony skiff'), 'green', None, 3, ('sabres', 5), 99))
decker.append((('ship', 'zielony skiff'), 'green', None, 4, ('sabres', 5), 100))
decker.append((('ship', 'czerwona fregata'), 'red', None, 1, ('sabres', 1), 101))
decker.append((('ship', 'czerwona fregata'), 'red', None, 1, ('sabres', 1), 102))
decker.append((('ship', 'czerwona fregata'), 'red', None, 1, ('sabres', 1), 103))
decker.append((('ship', 'czerwona fregata'), 'red', None, 2, ('sabres', 3), 104))
decker.append((('ship', 'czerwona fregata'), 'red', None, 2, ('sabres', 3), 105))
decker.append((('ship', 'czerwona fregata'), 'red', None, 2, ('sabres', 3), 106))
decker.append((('ship', 'czerwona fregata'), 'red', None, 3, ('sabres', 6), 107))
decker.append((('ship', 'czerwona fregata'), 'red', None, 3, ('sabres', 6), 108))
decker.append((('ship', 'czerwona fregata'), 'red', None, 3, ('sabres', 999), 109))
decker.append((('ship', 'czerwona fregata'), 'red', None, 4, ('sabres', 999), 110))
decker.append((('ship', 'czarny galeon'), 'black', None, 1, ('sabres', 2), 111))
decker.append((('ship', 'czarny galeon'), 'black', None, 1, ('sabres', 2), 112))
decker.append((('ship', 'czarny galeon'), 'black', None, 1, ('sabres', 2), 113))
decker.append((('ship', 'czarny galeon'), 'black', None, 2, ('sabres', 4), 114))
decker.append((('ship', 'czarny galeon'), 'black', None, 2, ('sabres', 4), 115))
decker.append((('ship', 'czarny galeon'), 'black', None, 2, ('sabres', 4), 116))
decker.append((('ship', 'czarny galeon'), 'black', None, 3, ('sabres', 7), 117))
decker.append((('ship', 'czarny galeon'), 'black', None, 3, ('sabres', 7), 118))
decker.append((('ship', 'czarny galeon'), 'black', None, 3, ('sabres', 999), 119))
decker.append((('ship', 'czarny galeon'), 'black', None, 4, ('sabres', 999), 120))

with open('pr_deck.txt', 'w') as f:
    json.dump(tuple(decker), f)

In [139]:
with open('pr_deck.txt', 'r') as f:
    decc = json.load(f)
decctmp=[]
decctuple=[]
for li in decc:
    for idx in range(len(li)):
        if type(li[idx]) == list:
            li[idx]=tuple(li[idx])
    decc[decc.index(li)] = tuple(li)
print(tuple(decc))

((('vendor', 'handlarz'), 'yellow', 1, 3, ('yellow_more', 1), 1), (('vendor', 'handlarz'), 'yellow', 2, 5, ('yellow_more', 1), 2), (('vendor', 'handlarz'), 'blue', 1, 3, ('blue_more', 1), 3), (('vendor', 'handlarz'), 'blue', 2, 5, ('blue_more', 1), 4), (('vendor', 'handlarz'), 'green', 1, 3, ('green_more', 1), 5), (('vendor', 'handlarz'), 'green', 1, 3, ('green_more', 1), 6), (('vendor', 'handlarz'), 'red', 1, 3, ('red_more', 1), 7), (('vendor', 'handlarz'), 'red', 1, 3, ('red_more', 1), 8), (('vendor', 'handlarz'), 'black', 1, 3, ('black_more', 1), 9), (('vendor', 'handlarz'), 'black', 1, 3, ('black_more', 1), 10), (('settler', 'osadnik'), None, 1, 4, ('houses', 1), 11), (('settler', 'osadnik'), None, 1, 4, ('houses', 1), 12), (('settler', 'osadnik'), None, 1, 4, ('houses', 1), 13), (('settler', 'osadnik'), None, 1, 4, ('houses', 1), 14), (('settler', 'osadnik'), None, 1, 4, ('houses', 1), 15), (('captain', 'kapitan'), None, 1, 4, ('anchors', 1), 16), (('captain', 'kapitan'), None, 1,

In [96]:
with open('game_settings.ini', 'r') as f:
    games = json.load(f)

for k,v in games.items():
    if type(v) == list:
        games[k]=tuple(v)
print(type(games))

<class 'dict'>


In [65]:
abb=[1,2,3,4,'a',5]
abb[abb.index('a')] = 'b'
abb

[1, 2, 3, 4, 'b', 5]

import json
deck2 = []
deck3 = {}

deck2.append(['vendor', u'handlarz', 'yellow', 1, 3, 12, 1])
deck2.append(['vendor', u'handlarz', 'yellow', 2, 5, 12, 2])
deck2.append(['vendor', u'handlarz', 'blue', 1, 3, 12, 3])
deck2.append(['vendor', u'handlarz', 'blue', 2, 5, 12, 4])
deck2.append(['vendor', u'handlarz', 'green', 1, 3, 12, 5])
deck2.append(['vendor', u'handlarz', 'green', 1, 3, 12, 6])
deck2.append(['vendor', u'handlarz', 'red', 1, 3, 12, 7])
deck2.append(['vendor', u'handlarz', 'red', 1, 3, 12, 8])
deck2.append(['vendor', u'handlarz', 'black', 1, 3, 12, 9])
deck2.append(['vendor', u'handlarz', 'black', 1, 3, 12, 10])

# Osadnicy (5)
deck2.append(['settler', u'osadnik', None, 1, 4, 8, 11])
deck2.append(['settler', u'osadnik', None, 1, 4, 8, 12])
deck2.append(['settler', u'osadnik', None, 1, 4, 8, 13])
deck2.append(['settler', u'osadnik', None, 1, 4, 8, 14])
deck2.append(['settler', u'osadnik', None, 1, 4, 8, 15])

# Kapitanowie (5)
deck2.append(['captain', u'kapitan', None, 1, 4, 9, 16])
deck2.append(['captain', u'kapitan', None, 1, 4, 9, 17])
deck2.append(['captain', u'kapitan', None, 1, 4, 9, 18])
deck2.append(['captain', u'kapitan', None, 1, 4, 9, 19])
deck2.append(['captain', u'kapitan', None, 1, 4, 9, 20])

# Kaplani (5)
deck2.append(['priest', u'kaplan', None, 1, 4, 10, 21])
deck2.append(['priest', u'kaplan', None, 1, 4, 10, 22])
deck2.append(['priest', u'kaplan', None, 1, 4, 10, 23])
deck2.append(['priest', u'kaplan', None, 1, 4, 10, 24])
deck2.append(['priest', u'kaplan', None, 1, 4, 10, 25])

# Zlote raczki (3)
deck2.append(['handyman', u'złota rączka', None, 1, 6, 11, 26])
deck2.append(['handyman', u'złota rączka', None, 1, 6, 11, 27])
deck2.append(['handyman', u'złota rączka', None, 1, 6, 11, 28])

# Zeglarze (10 - 7/2/1)
deck2.append(['sailor', u'żeglarz', None, 1, 3, 1, 29])
deck2.append(['sailor', u'żeglarz', None, 1, 3, 1, 30])
deck2.append(['sailor', u'żeglarz', None, 1, 3, 1, 31])
deck2.append(['sailor', u'żeglarz', None, 1, 3, 1, 32])
deck2.append(['sailor', u'żeglarz', None, 1, 3, 1, 33])
deck2.append(['sailor', u'żeglarz', None, 1, 3, 1, 34])
deck2.append(['sailor', u'żeglarz', None, 1, 3, 1, 35])
deck2.append(['sailor', u'żeglarz', None, 2, 5, 1, 36])
deck2.append(['sailor', u'żeglarz', None, 2, 5, 1, 37])
deck2.append(['sailor', u'żeglarz', None, 3, 7, 1, 38])

# Piraci (3 - 1/1/1)
deck2.append(['pirate', u'pirat', None, 1, 5, 2, 39])
deck2.append(['pirate', u'pirat', None, 2, 7, 2, 40])
deck2.append(['pirate', u'pirat', None, 3, 9, 2, 41])

# Madmoiselle (4 - 2/2)
deck2.append(['madmoiselle', u'madmoiselle', None, 2, 7, 13, 42])
deck2.append(['madmoiselle', u'madmoiselle', None, 2, 7, 13, 43])
deck2.append(['madmoiselle', u'madmoiselle', None, 3, 9, 13, 44])
deck2.append(['madmoiselle', u'madmoiselle', None, 3, 9, 13, 45])

# Blazen (5 - 1/3/1)
deck2.append(['jester', u'błazen', None, 1, 5, 14, 46])
deck2.append(['jester', u'błazen', None, 2, 7, 14, 47])
deck2.append(['jester', u'błazen', None, 2, 7, 14, 48])
deck2.append(['jester', u'błazen', None, 2, 7, 14, 49])
deck2.append(['jester', u'błazen', None, 3, 9, 14, 50])

# Armiral (6 - 1/3/2)
deck2.append(['admiral', u'admirał', None, 1, 5, 15, 51])
deck2.append(['admiral', u'admirał', None, 2, 7, 15, 52])
deck2.append(['admiral', u'admirał', None, 2, 7, 15, 53])
deck2.append(['admiral', u'admirał', None, 2, 7, 15, 54])
deck2.append(['admiral', u'admirał', None, 3, 9, 15, 55])
deck2.append(['admiral', u'admirał', None, 3, 9, 15, 56])

# Armiral (4)
deck2.append(['admiral', u'admirał', None, 0, 8, 16, 57])
deck2.append(['admiral', u'admirał', None, 0, 8, 16, 58])
deck2.append(['admiral', u'admirał', None, 0, 8, 16, 59])
deck2.append(['admiral', u'admirał', None, 0, 8, 16, 60])

# Ekspedycje (6)
deck2.append(['expedition', u'ekspedycja', None, 4, None, 17, 61])
deck2.append(['expedition', u'ekspedycja', None, 4, None, 18, 62])
deck2.append(['expedition', u'ekspedycja', None, 4, None, 19, 63])
deck2.append(['expedition', u'ekspedycja', None, 6, None, 20, 64])
deck2.append(['expedition', u'ekspedycja', None, 6, None, 21, 65])
deck2.append(['expedition', u'ekspedycja', None, 5, None, 22, 66])

# Podwyzka podatkow (4 - 2/2)
deck2.append(['tax', u'podwyżka podatków', None, None, None, 23, 67])
deck2.append(['tax', u'podwyżka podatków', None, None, None, 23, 68])
deck2.append(['tax', u'podwyżka podatków', None, None, None, 24, 69])
deck2.append(['tax', u'podwyżka podatków', None, None, None, 24, 70])

# Zolte statki (10)
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 1, 1, 71])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 1, 1, 72])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 1, 1, 73])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 2, 1, 74])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 2, 2, 75])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 2, 2, 76])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 3, 2, 77])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 3, 4, 78])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 3, 4, 79])
deck2.append(['ship', u'żółta szalupa', 'yellow', None, 4, 4, 80])

# Niebieskie statki (10)
deck2.append(['ship', u'niebieska fluita', 'blue', None, 1, 1, 81])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 1, 1, 82])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 1, 1, 83])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 2, 1, 84])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 2, 2, 85])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 2, 2, 86])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 3, 2, 87])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 3, 5, 88])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 3, 5, 89])
deck2.append(['ship', u'niebieska fluita', 'blue', None, 4, 5, 90])

# Zielone statki (10)
deck2.append(['ship', u'zielony skiff', 'green', None, 1, 1, 91])
deck2.append(['ship', u'zielony skiff', 'green', None, 1, 1, 92])
deck2.append(['ship', u'zielony skiff', 'green', None, 1, 1, 93])
deck2.append(['ship', u'zielony skiff', 'green', None, 2, 1, 94])
deck2.append(['ship', u'zielony skiff', 'green', None, 2, 3, 95])
deck2.append(['ship', u'zielony skiff', 'green', None, 2, 3, 96])
deck2.append(['ship', u'zielony skiff', 'green', None, 3, 3, 97])
deck2.append(['ship', u'zielony skiff', 'green', None, 3, 5, 98])
deck2.append(['ship', u'zielony skiff', 'green', None, 3, 5, 99])
deck2.append(['ship', u'zielony skiff', 'green', None, 4, 5, 100])

# Czerwone statki (10)
deck2.append(['ship', u'czerwona fregata', 'red', None, 1, 1, 101])
deck2.append(['ship', u'czerwona fregata', 'red', None, 1, 1, 102])
deck2.append(['ship', u'czerwona fregata', 'red', None, 1, 1, 103])
deck2.append(['ship', u'czerwona fregata', 'red', None, 2, 3, 104])
deck2.append(['ship', u'czerwona fregata', 'red', None, 2, 3, 105])
deck2.append(['ship', u'czerwona fregata', 'red', None, 2, 3, 106])
deck2.append(['ship', u'czerwona fregata', 'red', None, 3, 6, 107])
deck2.append(['ship', u'czerwona fregata', 'red', None, 3, 6, 108])
deck2.append(['ship', u'czerwona fregata', 'red', None, 3, 0, 109])
deck2.append(['ship', u'czerwona fregata', 'red', None, 4, 0, 110])

# Czarene statki (10)
deck2.append(['ship', u'czarny galeon', 'black', None, 1, 2, 111])
deck2.append(['ship', u'czarny galeon', 'black', None, 1, 2, 112])
deck2.append(['ship', u'czarny galeon', 'black', None, 1, 2, 113])
deck2.append(['ship', u'czarny galeon', 'black', None, 2, 4, 114])
deck2.append(['ship', u'czarny galeon', 'black', None, 2, 4, 115])
deck2.append(['ship', u'czarny galeon', 'black', None, 2, 4, 116])
deck2.append(['ship', u'czarny galeon', 'black', None, 3, 7, 117])
deck2.append(['ship', u'czarny galeon', 'black', None, 3, 7, 118])
deck2.append(['ship', u'czarny galeon', 'black', None, 3, 0, 119])
deck2.append(['ship', u'czarny galeon', 'black', None, 4, 0, 120])

deck3 = {}
for idx in deck2:
    deck3[idx[-1]] = idx

with open('deck_portroyal.txt', 'w') as f:
    json.dump(deck2, f)

with open('deck_portroyal.txt', 'r') as f:
    deck4 = json.load(f)